In [1]:
import pandas as pd
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import math
import re
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option('display.max_colwidth', None)
orari = pd.date_range(start="09:00", end="20:00", freq="5min").strftime('%H:%M')
tatami = ['Tatami ' + str(i) for i in range(1,7)]
timetable_sabato = pd.DataFrame(index=orari, columns=tatami)
timetable_domenica = pd.DataFrame(index=orari, columns=tatami)
timetable_sabato = pd.DataFrame(index=orari, columns=tatami).fillna('')
timetable_domenica = pd.DataFrame(index=orari, columns=tatami).fillna('')
timetable_sabato['Disponibile'] = True
timetable_domenica['Disponibile'] = True


In [2]:
iscrizioni = pd.read_parquet("iscrizioni.parquet")
iscrizioni["N"] = iscrizioni["N"].astype(int)
iscrizioni["Campo9"] = iscrizioni["Campo9"].astype(int)
iscrizioni["N CLUB"] = iscrizioni["N CLUB"].astype(int)
iscrizioni["SESSO"] = iscrizioni["SESSO"].str.upper()
iscrizioni["Kyu/Kup"] = iscrizioni["Kyu/Kup"].astype(int)
iscrizioni["Dan"] = iscrizioni["Dan"].astype(int)
iscrizioni["PESO / KG"] = iscrizioni["PESO / KG"].astype(int)
iscrizioni["ALTEZZA / CM"] = iscrizioni["ALTEZZA / CM"].astype(int)
iscrizioni["DATA DI NASCITA (AAAA-MM-GG)"] = pd.to_datetime(iscrizioni["DATA DI NASCITA (AAAA-MM-GG)"]).dt.date
iscrizioni.rename(columns={
    'NAZIONALITÀ': 'NAZIONALITA', 
    'N': 'ID_ATLETA',
    'DATA DI NASCITA (AAAA-MM-GG)': 'DATA_DI_NASCITA',
    'Campo9': 'ID_CATEGORIA',
    'N CLUB': 'ID_CLUB',
    'NATIONAL ID': 'ID_NAZIONALE',
    'PESO / KG': 'PESO_KG',
    'ALTEZZA / CM': 'ALTEZZA_CM',
    'Kyu/Kup': 'KYU_KUP',
    'Dan': 'DAN',
    'TESTA DI SERIE': 'TESTA_DI_SERIE'
}, inplace=True)

In [3]:
iscrizioni

,WKF-ID,NAZIONALITA,CLUB,NAZIONALE,ID_ATLETA,NOME,COGNOME,DATA_DI_NASCITA,ID_CATEGORIA,CATEGORIA,SQUADRE,STATUS,COMMENTO,TEMPO,FOTO,ID PASSAPORTO,ACCREDITO STAMPA,ACCREDITO TEMPO,SELEZIONARE LE AREE DI ACCESSO,Campo20,CONTROLLO PESO / ALTEZZA,CONTROLLO MEDICO,TABELLONI,ID_CLUB,SESSO,KYU_KUP,DAN,DEPENNARE,TESTA_DI_SERIE,ID_NAZIONALE,PESO_KG,ALTEZZA_CM
0,NaN,AUSTRIA,KADER_KARATE_TIROL(KADER_KT),None,930832,CLEA,STEIN,2003-08-07,122604,ESKA INDIVIDUAL KATA FEMALE JUNIOR BROWN,None,NaN,NaN,2023-2-22 08:30:38,NaN,None,1.0,2023-2-25 12:36:02,0.0,NaN,NaN,NaN,NaN,52053,F,2,0,NaN,NaN,None,50,0
1,NaN,AUSTRIA,KADER_KARATE_TIROL(KADER_KT),None,930825,MARIALENA,ALDEMCHKI,2010-11-14,122586,ESKA INDIVIDUAL KATA FEMALE KIDS GREEN - BLUE,None,NaN,NaN,2023-2-22 08:30:37,NaN,None,1.0,2023-2-24 10:16:11,0.0,NaN,NaN,NaN,NaN,52053,F,4,0,NaN,NaN,None,50,0
2,NaN,AUSTRIA,KADER_KARATE_TIROL(KADER_KT),None,930829,REGINA,PLATTNER,2010-05-28,122586,ESKA INDIVIDUAL KATA FEMALE KIDS GREEN - BLUE,None,NaN,NaN,2023-2-22 08:30:35,NaN,None,1.0,2023-2-24 10:16:11,0.0,NaN,NaN,NaN,NaN,52053,F,6,0,NaN,NaN,None,50,0
3,NaN,AUSTRIA,KADER_KARATE_TIROL(KADER_KT),None,930824,DANIELLA,ALDEMCHKI,2008-01-05,122596,ESKA INDIVIDUAL KATA FEMALE YOUTH GREEN -BLUE,None,NaN,NaN,2023-2-22 08:30:37,NaN,None,1.0,2023-2-24 10:16:11,0.0,NaN,NaN,NaN,NaN,52053,F,4,0,NaN,NaN,None,50,0
4,NaN,AUSTRIA,KADER_KARATE_TIROL(KADER_KT),None,930831,AYLIN,SCHMID,2009-12-15,122596,ESKA INDIVIDUAL KATA FEMALE YOUTH GREEN -BLUE,None,NaN,NaN,2023-2-22 08:30:34,NaN,None,1.0,2023-2-24 10:16:11,0.0,NaN,NaN,NaN,NaN,52053,F,5,0,NaN,NaN,None,50,0
5,NaN,AUSTRIA,KADER_KARATE_TIROL(KADER_KT),None,930834,TOBIAS,FALKNER,2003-08-24,122601,ESKA INDIVIDUAL KATA MALE JUNIOR BROWN,None,NaN,NaN,2023-2-22 08:30:37,NaN,None,1.0,2023-2-24 10:16:11,0.0,NaN,NaN,NaN,NaN,52053,M,1,0,NaN,NaN,None,50,0
6,NaN,AUSTRIA,KADER_KARATE_TIROL(KADER_KT),None,930833,SANDRO,WIESER,2004-06-01,122601,ESKA INDIVIDUAL KATA MALE JUNIOR BROWN,None,NaN,NaN,2023-2-22 08:30:38,NaN,None,1.0,2023-2-25 12:02:14,0.0,NaN,NaN,NaN,NaN,52053,M,1,0,NaN,NaN,None,50,0
7,NaN,AUSTRIA,KADER_KARATE_TIROL(KADER_KT),None,930827,BEN,KIRCHEBNER,2010-08-19,122589,ESKA INDIVIDUAL KATA MALE KIDS GREEN - BLUE,None,NaN,NaN,2023-2-22 08:30:36,NaN,None,1.0,2023-2-24 10:16:11,0.0,NaN,NaN,NaN,NaN,52053,M,6,0,NaN,NaN,None,50,0
8,NaN,AUSTRIA,KADER_KARATE_TIROL(KADER_KT),None,930828,CONSTANTIN,MOESL,2010-01-08,122589,ESKA INDIVIDUAL KATA MALE KIDS GREEN - BLUE,None,NaN,NaN,2023-2-22 08:30:36,NaN,None,1.0,2023-2-24 10:16:11,0.0,NaN,NaN,NaN,NaN,52053,M,4,0,NaN,NaN,None,50,0
9,NaN,AUSTRIA,KADER_KARATE_TIROL(KADER_KT),None,930830,MAGNUS,RANHARTSTETTER,2009-03-10,122593,ESKA INDIVIDUAL KATA MALE YOUTH GREEN - BLUE,None,NaN,NaN,2023-2-22 08:30:35,NaN,None,1.0,2023-2-28 17:59:28,0.0,NaN,NaN,NaN,NaN,52053,M,6,0,NaN,NaN,None,50,0


In [4]:
iscrizioni.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1504 entries, 0 to 1503
Data columns (total 32 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   WKF-ID                          0 non-null      float64
 1   NAZIONALITA                     1504 non-null   object 
 2   CLUB                            1504 non-null   object 
 3   NAZIONALE                       503 non-null    object 
 4   ID_ATLETA                       1504 non-null   int32  
 5   NOME                            1504 non-null   object 
 6   COGNOME                         1504 non-null   object 
 7   DATA_DI_NASCITA                 1504 non-null   object 
 8   ID_CATEGORIA                    1504 non-null   int32  
 9   CATEGORIA                       1504 non-null   object 
 10  SQUADRE                         328 non-null    object 
 11  STATUS                          0 non-null      float64
 12  COMMENTO                        0 

In [5]:
atleti = iscrizioni.drop_duplicates(subset=['ID_ATLETA']).loc[:, ["ID_ATLETA", "COGNOME", "NOME", "DATA_DI_NASCITA", "SESSO", "KYU_KUP", "DAN", "PESO_KG", "ALTEZZA_CM", "NAZIONALITA", "CLUB", "TESTA_DI_SERIE"]].sort_values(by=['COGNOME', 'NOME'], ascending=True)
atleti.reset_index(drop=True, inplace=True)
atleti.set_index("ID_ATLETA", drop=True, inplace=True)
atleti

,COGNOME,NOME,DATA_DI_NASCITA,SESSO,KYU_KUP,DAN,PESO_KG,ALTEZZA_CM,NAZIONALITA,CLUB,TESTA_DI_SERIE
ID_ATLETA,,,,,,,,,,,
757478,ABDO,WILLIAM,2007-01-30,M,0,0,0,0,ISRAEL,REN BU KAN KARATE NAZARETH(RBKN),NaN
812047,ABU,GABRIELE,2008-01-02,M,0,1,70,0,ITALY,ASD YAMA KARATE(YAMA),NaN
927967,ABU_AHMAD,MARWAN,2005-01-01,M,0,0,61,0,ISRAEL,REN BU KAN KARATE NAZARETH(RBKN),NaN
606008,ABU_ASSAD,ADAN,2006-09-02,F,0,0,45,0,ISRAEL,REN BU KAN KARATE NAZARETH(RBKN),NaN
812935,ACCARDI,ANDREA,2004-08-23,M,0,1,62,160,ITALY,SSD MOVING CLUB CREMA(MOVING CLUB),NaN
930483,ACCILI,ALICE,2011-12-30,F,4,0,31,141,ITALY,MOSTARDA(MOSTARDA),NaN
927983,ADAM,ABU_ASAD,2011-01-01,M,0,0,0,0,ISRAEL,REN BU KAN KARATE NAZARETH(RBKN),NaN
927984,ADAM,MARSHY,2011-01-01,M,0,0,0,0,ISRAEL,REN BU KAN KARATE NAZARETH(RBKN),NaN
673725,AGOSTINI,FRANCESCA,2002-03-11,F,0,1,59,2,ITALY,NAKA(NAKA),NaN


In [6]:
atleti.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 900 entries, 757478 to 608943
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   COGNOME          900 non-null    object 
 1   NOME             900 non-null    object 
 2   DATA_DI_NASCITA  900 non-null    object 
 3   SESSO            899 non-null    object 
 4   KYU_KUP          900 non-null    int32  
 5   DAN              900 non-null    int32  
 6   PESO_KG          900 non-null    int32  
 7   ALTEZZA_CM       900 non-null    int32  
 8   NAZIONALITA      900 non-null    object 
 9   CLUB             900 non-null    object 
 10  TESTA_DI_SERIE   0 non-null      float64
dtypes: float64(1), int32(4), object(6)
memory usage: 70.3+ KB


In [7]:
club = iscrizioni[['ID_CLUB', 'CLUB']].drop_duplicates().reset_index(drop=True)
club.set_index("ID_CLUB", drop=True, inplace=True)
conteggio_atleti = iscrizioni.groupby('CLUB')['ID_ATLETA'].count().reset_index(name='TOT_ATLETI')
lista_atleti = iscrizioni.groupby('CLUB')['ID_ATLETA'].apply(list).reset_index(name='LISTA_ATLETI')
club_info = pd.merge(conteggio_atleti, lista_atleti, on='CLUB')
club = pd.merge(club, club_info, on='CLUB', how='left')
club

,CLUB,TOT_ATLETI,LISTA_ATLETI
0,KADER_KARATE_TIROL(KADER_KT),13,"[930832, 930825, 930829, 930824, 930831, 930834, 930833, 930827, 930828, 930830, 930826, 930832, 930833]"
1,BELGIAN AMATEUR KARATE FEDERATION(BELGIUM),36,"[928970, 928971, 709182, 810698, 810697, 812162, 928972, 633085, 810694, 564915, 497733, 808742, 928970, 928971, 928970, 928971, 709182, 709182, 810698, 810697, 812162, 810698, 810697, 812162, 633085, 810694, 564915, 497733, 808742, 928972, 633085, 810694, 808742, 928972, 564915, 497733]"
2,SK SHOTOKAN NERATOVICE(SHOTOKAN NERATOVICE),18,"[608943, 930124, 608943, 930124, 930124, 608943, 930127, 930127, 608950, 608977, 608976, 608954, 673008, 608950, 608977, 608976, 608954, 673008]"
3,MAKHARADZE(MAKHARADZE),3,"[926339, 926339, 926339]"
4,KARATE DOJO DJK-SPORTFREUNDE REICHENBERG(KARATE DOJO REICHENBERG),14,"[765281, 761290, 779772, 761289, 763271, 829735, 829737, 929786, 929787, 763161, 763160, 756036, 761288, 793114]"
5,SANTORO(SANTORO),1,[921912]
6,REN BU KAN KARATE NAZARETH(RBKN),38,"[606008, 927986, 529214, 595820, 927983, 927984, 927985, 757477, 927978, 606008, 529205, 529214, 606008, 529205, 529214, 927987, 757478, 927972, 595820, 927968, 595821, 595820, 927968, 757478, 927972, 595821, 927967, 927967, 594043, 594043, 927981, 757477, 927977, 927976, 927980, 757478, 595820, 595821]"
7,A.K. - MILANO(A.K. - MILANO),6,"[810000, 809999, 810004, 810003, 809998, 810001]"
8,A.S.D. BUTOKU KARATE LOMAZZO(BUTOKU LOMAZZO),5,"[810384, 810384, 810384, 576436, 576436]"
9,A.S.D. DOJO SHOTOKAN KARATE(DOJO SHOTOKAN TREVISO),6,"[812885, 930759, 812883, 817202, 817199, 817203]"


In [8]:
club.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 106 entries, 0 to 105
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   CLUB          106 non-null    object
 1   TOT_ATLETI    106 non-null    int64 
 2   LISTA_ATLETI  106 non-null    object
dtypes: int64(1), object(2)
memory usage: 3.3+ KB


In [18]:
iscrizioni_filtrate = iscrizioni.dropna(subset=['NAZIONALE'])
nazionali = iscrizioni_filtrate[['NAZIONALE']].drop_duplicates().reset_index(drop=True)
nazionali.set_index("NAZIONALE", drop=True, inplace=True)
conteggio_atleti = iscrizioni_filtrate.groupby('NAZIONALE')['ID_ATLETA'].count().reset_index(name='TOT_ATLETI')
lista_atleti = iscrizioni_filtrate.groupby('NAZIONALE')['ID_ATLETA'].apply(list).reset_index(name='LISTA_ATLETI')
nazionali_info = pd.merge(conteggio_atleti, lista_atleti, on='NAZIONALE')
nazionali = pd.merge(nazionali, nazionali_info, on='NAZIONALE', how='left')
nazionali

,NAZIONALE,TOT_ATLETI,LISTA_ATLETI
0,03 LOM,231,"[810384, 810384, 810384, 576436, 576436, 812197, 811677, 812197, 812197, 811677, 811677, 896073, 896073, 813138, 813140, 811997, 927461, 734995, 812004, 927466, 812013, 927468, 927469, 927476, 594255, 812014, 735000, 593777, 593766, 734996, 756051, 812012, 811995, 812009, 811986, 811988, 927464, 811992, 928733, 928734, 811987, 927477, 927475, 594281, 927474, 812932, 734997, 809320, 809319, 927473, 927467, 593777, 593766, 756051, 811997, 812004, 927461, 734995, 593777, 593768, 593766, 734996, 756051, 811997, 812004, 927461, 812013, 734995, 811997, 594255, 593766, 593766, 593777, 734996, 735000, 812006, 928731, 811995, 812009, 812006, 594281, 734997, 927465, 809320, 734997, 927465, 927467, 734997, 809320, 809319, 811995, 928731, 812000, 734997, 927465, 809320, 809319, 931081, 813384, 813383, ...]"
1,05 VEN,127,"[812885, 930759, 812883, 817202, 817199, 817203, 930897, 931038, 930894, 930890, 930837, 930892, 930895, 930904, 930896, 930893, 931036, 930487, 930721, 930488, 772055, 805733, 925601, 930957, 595045, 930502, 732981, 930720, 594388, 594388, 593629, 752335, 812185, 812185, 812184, 930959, 812184, 930959, 930958, 812081, 800899, 800899, 925414, 928143, 928143, 800900, 800896, 800900, 597904, 597905, 928513, 928514, 928510, 597903, 597904, 597905, 928515, 597903, 808520, 756766, 808521, 756767, 808518, 756756, 756763, 756762, 756765, 808527, 808522, 808523, 808519, 808517, 808526, 756759, 756757, 756761, 756756, 756763, 756762, 808517, 756761, 756759, 756761, 756759, 756762, 862258, 700045, 810482, 859632, 810482, 700049, 503087, 503087, 810480, 929739, 856882, 929739, 859632, 856882, 810480, ...]"
2,09 TOS,53,"[757068, 757068, 634427, 634427, 757102, 599309, 929771, 929772, 757066, 929771, 929772, 757066, 599309, 463968, 930347, 930347, 463968, 757071, 599309, 929771, 929772, 757066, 809250, 926951, 926949, 809260, 809261, 809262, 809233, 809245, 809230, 809231, 809244, 809249, 809250, 809260, 809261, 809233, 809230, 809244, 809231, 809249, 727846, 929031, 930396, 930396, 929031, 815162, 815162, 930394, 930394, 930708, 929032]"
3,01 PIE,26,"[727248, 727248, 727248, 812746, 928580, 928582, 928596, 812739, 928584, 928585, 812745, 928595, 812740, 928594, 928588, 928593, 928589, 812119, 927142, 927139, 812118, 813358, 927140, 812119, 597472, 597472]"
4,06 FRI,7,"[871473, 861789, 861788, 929648, 929535, 861788, 861789]"
5,08 EMI,59,"[931127, 931126, 931129, 931141, 931026, 931028, 931024, 931141, 931026, 931028, 931024, 931144, 931027, 931025, 927373, 927358, 927364, 928333, 928322, 928336, 813191, 928340, 813192, 598037, 505693, 505694, 924127, 928335, 597907, 928304, 928815, 928333, 928336, 928322, 928340, 598037, 598037, 505693, 505694, 924127, 924127, 505693, 505694, 597907, 928335, 928304, 928815, 597907, 587353, 724722, 806599, 922909, 922911, 587312, 587353, 587353, 724722, 806599, 587312]"


In [10]:
nazioni = iscrizioni[['NAZIONALITA']].drop_duplicates().reset_index(drop=True)
conteggio_atleti = iscrizioni.groupby('NAZIONALITA')['ID_ATLETA'].count().reset_index(name='TOT_ATLETI')
nazioni = pd.merge(nazioni, conteggio_atleti, on='NAZIONALITA', how='left')
nazioni.set_index("NAZIONALITA", drop=True, inplace=True)
nazioni

,TOT_ATLETI
NAZIONALITA,
AUSTRIA,13
BELGIUM,36
CZECH REPUBLIC,18
GEORGIA,3
GERMANY,15
ISRAEL,38
ITALY,1381


In [19]:
iscrizioni_filtrate = iscrizioni.dropna(subset=['SQUADRE'])
squadre = iscrizioni_filtrate[['SQUADRE']].drop_duplicates().reset_index(drop=True)
squadre.set_index("SQUADRE", drop=True, inplace=True)
conteggio_atleti = iscrizioni_filtrate.groupby('SQUADRE')['ID_ATLETA'].count().reset_index(name='TOT_ATLETI')
lista_atleti = iscrizioni_filtrate.groupby('SQUADRE')['ID_ATLETA'].apply(list).reset_index(name='LISTA_ATLETI')
squadre_info = pd.merge(conteggio_atleti, lista_atleti, on='SQUADRE')
squadre = pd.merge(squadre, squadre_info, on='SQUADRE', how='left')
squadre

,SQUADRE,TOT_ATLETI,LISTA_ATLETI
0,Team_Karate_Tirol,3,"[930826, 930832, 930833]"
1,Ren Bu Kan Karate Nazareth,9,"[606008, 529205, 529214, 606008, 529205, 529214, 757478, 595820, 595821]"
2,Accademia Shirai Pistoia,4,"[599309, 929771, 929772, 757066]"
3,asd Ki Dojo Caldiero,3,"[812185, 812184, 930959]"
4,Asd Kyudokan nettuno,3,"[930711, 930709, 930710]"
5,ASD M. NAKAYAMA REGGIO EMILIA,10,"[811582, 930740, 732476, 732449, 811577, 732230, 732449, 811577, 812249, 732230]"
6,asd ken e zen i.n. Moruzzo,6,"[873357, 873360, 873361, 861943, 861944, 861945]"
7,ASD Yama Karate Club Milano,30,"[668697, 595127, 352745, 668683, 812059, 668699, 812064, 812053, 668697, 595127, 925000, 595122, 668690, 595124, 668681, 812069, 812070, 925001, 812066, 668695, 595124, 668685, 595134, 595142, 595122, 668690, 924965, 595122, 352745, 595124]"
8,CRV Fikta Veneto,6,"[931523, 931522, 597992, 932391, 931518, 931520]"
9,CRV Fikta Veneto 1,6,"[812193, 928599, 812195, 928604, 928606, 597984]"


In [14]:
categorie = iscrizioni.groupby(['CATEGORIA'])['NOME'].count().reset_index(name="ATLETI")
categorie['TIPO'] = categorie['CATEGORIA'].apply(lambda x: 'KATA' if 'KATA' in x else ('KUMITE' if 'KUMITE' in x else 'ALTRO'))
categorie['TEAM'] = categorie['CATEGORIA'].apply(lambda x: 'x' if 'TEAM' in x else '')
categorie['CINTURE'] = categorie['CATEGORIA'].apply(
    lambda x: 'NERE' if 'BLACK' in x else (
        'COLORATE' if re.search('YELLOW|ORANGE|GREEN|BLUE|BROWN', x) else 'TUTTE'
    )
)
categorie['KIDS'] = categorie['CATEGORIA'].apply(lambda x: 'x' if 'KIDS' in x else '')
categorie['KIDS'] = categorie['CATEGORIA'].apply(lambda x: 'x' if 'KIDS' in x else '')
def check_over(x):
    cleaned_x = re.sub(r'\bOVER A\b', '', x)
    return 'x' if re.search(r'\bOVER\b', cleaned_x) else ''

def check_over_a(x):
    return 'x' if re.search(r'\bOVER A\b', x) else ''
categorie['OVER'] = categorie['CATEGORIA'].apply(check_over)
categorie['OVER A'] = categorie['CATEGORIA'].apply(check_over_a)


In [15]:
over32 = [
    (1, 33),
    (17, 49),
    (9, 41),
    (25, 57),
    (5, 37),
    (21, 53),
    (13, 45),
    (29, 61),
    (3, 35),
    (19, 51),
    (11, 43),
    (27, 59),
    (7, 39),
    (23, 55),
    (15, 47),
    (31, 63),
    (2, 34),
    (18, 50),
    (10, 42),
    (26, 58),
    (6, 38),
    (22, 54),
    (14, 46),
    (30, 62),
    (4, 36),
    (20, 52),
    (12, 44),
    (28, 60),
    (8, 40),
    (24, 56),
    (16, 48),
    (32, 64)
]

sedicesimi = [
    (1, 17),
    (9, 25),
    (5, 21), 
    (13, 29),
    (3, 19),
    (11, 27),
    (7, 23),
    (15, 31),
    (2,  18),
    (10, 26),
    (6, 22),
    (14, 30),
    (4, 20),
    (12, 28),
    (8, 24),
    (16, 32)
]

ottavi = [
    (1, 9),
    (5, 13),
    (3, 11),
    (7, 15),
    (2, 10),
    (6, 14),
    (4, 12),
    (8, 16)
]

quarti = [
    (1, 5),
    (3, 7),
    (2, 6),
    (4, 8)
]

semifinale = [
    (1, 3), 
    (2, 4)
]

finale = [
    (1, 2)
]



In [16]:
# Variabili di tempo in minuti
tempo_kata_kids_over = 2 #da moltiplicare per 2 prove
tempo_kata_nere_bandierine = 2
tempo_kata_nere_punteggio = 2.5
tempo_kata_colorate = 2.5
tempo_kata_team = 2.5
tempo_kumite = 3.5  

# Funzione per calcolare il tempo totale per ciascuna categoria
def calcola_tempo(row):
    # KATA
    if row["TIPO"] == "KATA":
        # SQUADRE
        if row["TEAM"] == "x":
            return 0
        # INDIVIDUALI 
        else:
            # KIDS, OVER e OVER A a punteggio
            if row["KIDS"] == "x" or row["OVER"] == "x" or row["OVER A"] == "x":
                return math.ceil(row["ATLETI"] * tempo_kata_kids_over)
            # ALTRE ETA' CINTURE COLORATE
            elif row["CINTURE"] == "COLORATE":
                return math.ceil(row["ATLETI"] * tempo_kata_colorate)
            # ALTRE ETA' CINTURE NERE
            elif row["CINTURE"] == "NERE":
                n_atleti = row["ATLETI"]            
                tempo = 0
                # Più di 32 atleti, vedi prima colonna del tabellone gare - BANDIERINE
                if n_atleti > 32:
                    incontri_over32 = [(a, b) for a, b in over32 if a <= n_atleti and b <= n_atleti]
                    n_incontri = len(incontri_over32)
                    tempo += n_incontri * tempo_kata_nere_bandierine
                    n_atleti = 32
                # Fra 32 e 16 atleti, vedi colonna "sedicesimi" - BANDIERINE
                if n_atleti > 16:
                    incontri_sedicesimi = [(a, b) for a, b in sedicesimi if a <= n_atleti and b <= n_atleti]
                    n_incontri = len(incontri_sedicesimi)
                    tempo += n_incontri * tempo_kata_nere_bandierine
                    n_atleti = 16
                # Fra 16 e 8 atleti, vedi colonna "ottavi" - BANDIERINE
                if n_atleti > 8:
                    incontri_ottavi = [(a, b) for a, b in ottavi if a <= n_atleti and b <= n_atleti]
                    n_incontri = len(incontri_ottavi)
                    tempo += n_incontri * tempo_kata_nere_bandierine
                    n_atleti = 8
                # Fra 8 e 4 atleti, vedi colonna "quarti" - PUNTEGGIO
                if n_atleti > 4:
                    n_incontri = n_atleti # "incontri singoli", ogni atleta gareggia da solo
                    tempo += n_incontri * tempo_kata_nere_punteggio
                    n_atleti = 4
                # SEMIFINALE - PUNTEGGIO
                n_incontri = n_atleti # "incontri singoli", ogni atleta gareggia da solo
                tempo += n_incontri * tempo_kata_nere_punteggio
                n_atleti = 2
                # FINALE - PUNTEGGIO
                n_incontri = n_atleti # "incontri singoli", ogni atleta gareggia da solo
                tempo += n_incontri * tempo_kata_nere_punteggio                  

                return math.ceil(tempo)
            
            # ERRORE
            else:
                return -1
    # KUMITE
    else:
        # SQUADRE
        if row["TEAM"] == "x":
            return 0
        # INDIVIDUALI 
        else:
            n_atleti = row["ATLETI"]            
            tempo = 0
            # Più di 32 atleti, vedi prima colonna del tabellone gare 
            if n_atleti > 32:
                incontri_over32 = [(a, b) for a, b in over32 if a <= n_atleti and b <= n_atleti]
                n_incontri = len(incontri_over32)
                tempo += n_incontri * tempo_kumite
                n_atleti = 32
            # Fra 32 e 16 atleti, vedi colonna "sedicesimi"
            if n_atleti > 16:
                incontri_sedicesimi = [(a, b) for a, b in sedicesimi if a <= n_atleti and b <= n_atleti]
                n_incontri = len(incontri_sedicesimi)
                tempo += n_incontri * tempo_kumite
                n_atleti = 16
            # Fra 16 e 8 atleti, vedi colonna "ottavi"
            if n_atleti > 8:
                incontri_ottavi = [(a, b) for a, b in ottavi if a <= n_atleti and b <= n_atleti]
                n_incontri = len(incontri_ottavi)
                tempo += n_incontri * tempo_kumite
                n_atleti = 8
            # Fra 8 e 4 atleti, vedi colonna "quarti" 
            if n_atleti > 4:
                incontri_quarti = [(a, b) for a, b in quarti if a <= n_atleti and b <= n_atleti]
                n_incontri = len(incontri_quarti)
                tempo += n_incontri * tempo_kumite
                n_atleti = 4
            # SEMIFINALE 
            if n_atleti > 2:
                incontri_semifinale = [(a, b) for a, b in semifinale if a <= n_atleti and b <= n_atleti]
                n_incontri = len(incontri_semifinale)
                tempo += n_incontri * tempo_kumite
                n_atleti = 2
            # FINALE 
            tempo += tempo_kumite                  

            return math.ceil(tempo)
            
categorie['TEMPO (MINUTI)'] = categorie.apply(lambda row: calcola_tempo(row), axis=1)
categorie.set_index("CATEGORIA", drop=True, inplace=True)
categorie

,ATLETI,TIPO,TEAM,CINTURE,KIDS,OVER,OVER A,TEMPO (MINUTI)
CATEGORIA,,,,,,,,
ESKA INDIVIDUAL KATA FEMALE OVER+ OVER A YELLOW AND ORANGE,6,KATA,,COLORATE,,x,x,12
ESKA INDIVIDUAL KATA FEMALE CADETS BLACK BELT,28,KATA,,NERE,,,,75
ESKA INDIVIDUAL KATA FEMALE CADETS BROWN,19,KATA,,COLORATE,,,,48
ESKA INDIVIDUAL KATA FEMALE CADETS GREEN - BLUE,7,KATA,,COLORATE,,,,18
ESKA INDIVIDUAL KATA FEMALE JUNIOR BLACK BELT,33,KATA,,NERE,,,,85
ESKA INDIVIDUAL KATA FEMALE JUNIOR BROWN,6,KATA,,COLORATE,,,,15
ESKA INDIVIDUAL KATA FEMALE JUNIOR+SENIOR + OVER GREEN AND BLUE,11,KATA,,COLORATE,,x,,22
ESKA INDIVIDUAL KATA FEMALE KIDS BROWN + BLACK BELT,11,KATA,,NERE,x,,,22
ESKA INDIVIDUAL KATA FEMALE KIDS GREEN - BLUE,21,KATA,,COLORATE,x,,,42


In [17]:
categorie.info()

<class 'pandas.core.frame.DataFrame'>
Index: 87 entries, ESKA INDIVIDUAL KATA FEMALE   OVER+ OVER A YELLOW AND ORANGE to TEAM KUMITE FOR REPRESENTATIVES JUNIOR-SENIOR 3 MALE + 2 FEMALE
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ATLETI          87 non-null     int64 
 1   TIPO            87 non-null     object
 2   TEAM            87 non-null     object
 3   CINTURE         87 non-null     object
 4   KIDS            87 non-null     object
 5   OVER            87 non-null     object
 6   OVER A          87 non-null     object
 7   TEMPO (MINUTI)  87 non-null     int64 
dtypes: int64(2), object(6)
memory usage: 6.1+ KB
